In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [2]:
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
features = pd.read_csv('data_set.csv', low_memory = False)

In [3]:
print(training_labels.shape)
training_labels = pd.merge(features, training_labels, on='person', how='inner')
print(training_labels.shape)
training_labels.head(2)

(19414, 2)
(19414, 218)


,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,minimo,suma_resolution,promedio_resolution,maximo_resolution,minimo_resolution,promedio de TFIdf modelo,Suma de TFidf modelo,Maximo de TFIdf modelo,Minimo de TFIdf modelo,label
0,0008ed71,2.0,0.0,0.0,0.0,0.0,0.0,0,0,3,...,20180517122747,4147200,691200.000000,2073600,0,1.617163,4.851488,1.859879,1.300103,0
1,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,20180529002747,230400,13552.941176,230400,0,1.000000,4.000000,1.000000,1.000000,0


In [4]:
print(labels_predict.shape)
labels_predict = pd.merge(features, labels_predict, on='person', how='inner')
print(labels_predict.shape)
labels_predict.head(2)

(19415, 1)
(19415, 217)


,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,maximo,minimo,suma_resolution,promedio_resolution,maximo_resolution,minimo_resolution,promedio de TFIdf modelo,Suma de TFidf modelo,Maximo de TFIdf modelo,Minimo de TFIdf modelo
0,00091926,1.0,0.0,25.0,0.0,0.0,0.0,15,25,2,...,20180531195203,20180503220829,26738688,59684.571429,786432,0,1.490856,557.580165,2.209395,1.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0,...,20180326145658,20180326145111,230400,23040.000000,230400,0,1.219303,3.657909,1.357806,1.0


In [5]:
labels = training_labels['label']
features = training_labels.drop(['label','person'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  17472 Test:  1942


In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(x_train)
X_test_scale = scaler.transform(x_test)

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.05,0.1,],
    #"min_samples_split": np.linspace(0.1, 0.5, 12),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.8, 1.0],
    "n_estimators":[10]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=2, n_jobs=-1)

clf.fit(X_train_scale, y_train)
print(clf.score(X_train_scale, y_train))
print(clf.best_params_)

0.9493475274725275
{'criterion': 'friedman_mse', 'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 10, 'subsample': 0.5}


In [9]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

gb = GradientBoostingClassifier(criterion= 'friedman_mse',
                                learning_rate= 0.05, loss= 'deviance',
                                max_depth= 5, max_features= 'log2', 
                                n_estimators= 10, subsample= 0.5)
gb.fit(X_train_scale, y_train)
predictions = gb.predict(X_test_scale)

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print()
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix:
[[1847    0]
 [  95    0]]

Classification Report
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      1847
          1       0.00      0.00      0.00        95

avg / total       0.90      0.95      0.93      1942



/home/camila/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
# ROC curve and Area-Under-Curve (AUC)

y_scores_gb = gb.decision_function(X_test_scale)
fpr_gb, tpr_gb, _ = roc_curve(y_test, y_scores_gb)
roc_auc_gb = auc(fpr_gb, tpr_gb)

print("Area under ROC curve = {:0.2f}".format(roc_auc_gb))

Area under ROC curve = 0.82


In [11]:
set_test = labels_predict
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])
set_test = set_test.fillna(0)
set_test_scaled = scaler.fit_transform(set_test)


In [12]:
preds_kaggle_knn = gb.predict_proba(set_test_scaled)

In [13]:
df_kaggel_knn = pd.DataFrame(preds_kaggle_knn)
df_kaggel_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19415 entries, 0 to 19414
Data columns (total 2 columns):
0    19415 non-null float64
1    19415 non-null float64
dtypes: float64(2)
memory usage: 303.4 KB


In [14]:
df_kaggel_knn = pd.DataFrame(df_kaggel_knn[1])

In [15]:
df_kaggel_knn["person"] = personas
df_kaggel_knn.rename(columns = {1: 'label'},inplace = True) 
df_kaggel_knn = df_kaggel_knn[['person','label']]

In [16]:
df_kaggel_knn.head()

,person,label
0,00091926,0.054412
1,00091a7a,0.082344
2,000ba417,0.055227
3,000e4d9e,0.051375
4,000e619d,0.059201


In [17]:
df_kaggel_knn.to_csv('ResultadosGBC.csv',index=False)